In [1]:
!pip install pandas
!pip install scikit-learn
!pip install seaborn
!pip install lightgbm[scikit-learn]
!pip install plotly
!pip install xgboost
!pip install catboost
!pip install joblib
!pip install kaggle

zsh:1: no matches found: lightgbm[scikit-learn]


In [4]:
#imports
# Data Analytic Tools
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns 
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline
%config InlineBackend.figure_format='retina'

# Computing Tools 
import lightgbm as lgb 
import xgboost as xgb 
import catboost as cbt 
import numpy as np 
import joblib 
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


for dirname, _, filenames in os.walk('data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

data/__init__.py
data/public_timeseries_testing_util.py
data/train.csv
data/models/{modelname}_{i}.model
data/example_test_files/revealed_targets.csv
data/example_test_files/test.csv
data/example_test_files/sample_submission.csv
data/optiver2023/__init__.py
data/optiver2023/competition.cpython-310-x86_64-linux-gnu.so
data/optiver2023/__pycache__/__init__.cpython-311.pyc
data/.ipynb_checkpoints/public_timeseries_testing_util-checkpoint.py
data/catboost_info/learn_error.tsv
data/catboost_info/test_error.tsv
data/catboost_info/time_left.tsv
data/catboost_info/catboost_training.json
data/catboost_info/learn/events.out.tfevents
data/catboost_info/test/events.out.tfevents


In [5]:
train = pd.read_csv('data/train.csv')

train.head()

train.isnull().sum()

train = train.dropna(subset=['target'])
targets = train['target'].values
train = train.drop('time_id', axis=1)
train = train.drop('row_id', axis=1)
# Drop the index inplace 
train.reset_index(drop=True, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(train, targets, train_size=0.7, random_state=3)

In [6]:
from numba import njit, prange

def generate_features(df: pd.DataFrame) -> pd.DataFrame:
    features = ['seconds_in_bucket', 'imbalance_buy_sell_flag',
               'imbalance_size', 'matched_size', 'bid_size', 'ask_size',
                'reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap',
                'imb_s1', 'imb_s2'
               ]

    df['imb_s1'] = df.eval('(bid_size-ask_size)/(bid_size+ask_size)')
    df['imb_s2'] = df.eval('(imbalance_size-matched_size)/(matched_size+imbalance_size)')
    
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    return df[features]
    

In [7]:
TRAINING = True
if TRAINING:
   df_ = generate_features(X_train)

In [16]:
model_path ='data/RKCP0219'
os.makedirs('models', exist_ok=True)

N_fold = 2

if TRAINING:
    X = df_.values
    Y = y_train
    X = X[np.isfinite(Y)]
    Y = Y[np.isfinite(Y)]
    index = np.arange(len(X))
    
models = []
scores = pd.DataFrame(columns=['name', 'score'])

def train_and_test(model_dict, modelname='lgb'):
    if TRAINING:
        model = model_dict[modelname]
        model.fit(X[index%N_fold!=i], Y[index%N_fold!=i], 
                    eval_set=[(X[index%N_fold==i], Y[index%N_fold==i])], 
                    # verbose=10, 
                    # early_stopping_rounds=100
                    )
        models.append(model)
        joblib.dump(model, f'data/models/{modelname}_{i}.model')
        
        feat = generate_features(X_test) 
        # Make predictions using the trained models and compute the mean
        test_preds = model.predict(feat)
        test_score = mean_absolute_error(y_test, test_preds)
        scores.loc[len(scores)]=[modelname, test_score] 

    else:
        models.append(joblib.load(f'{model_path}/{modelname}_{i}.model'))
    return 

model_dict = {
    'lgb': lgb.LGBMRegressor(objective='regression_l1', n_estimators=500),
    'xgb': xgb.XGBRegressor(tree_method='hist', objective='reg:absoluteerror', n_estimators=500),
    'cbt': cbt.CatBoostRegressor(objective='MAE', iterations=10),
}

for i in range(N_fold):
    train_and_test(model_dict, 'lgb')
    train_and_test(model_dict, 'xgb')
    train_and_test(model_dict, 'cbt')

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.047386 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3118
[LightGBM] [Info] Number of data points in the train set: 1833262, number of used features: 14
[LightGBM] [Info] Start training from score -0.060201


KeyboardInterrupt: 

In [9]:
#Testing
scores

,name,score
0,cbt,6.420514
1,cbt,6.418807


In [10]:
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test() 

ModuleNotFoundError: No module named 'optiver2023'

In [ ]:
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    feat = generate_features(test)

    sample_prediction['target'] = np.mean([model.predict(feat) for model in models], 0)
    env.predict(sample_prediction)
    counter += 1